In [1]:
# 计算每个 component 参数的数量及未冻结参数占骨干网络的比例
import torch
# adapter_whole = '/path/to/codes/mmdet-VPT/work_dirs/FLIR/Ablation/flir_ablate_stage4/768_stage4_3-25/best_coco_bbox_mAP_iter_22000.pth'
# adapter_whole = torch.load(adapter_whole, map_location='cpu')

# ch6 = '/path/to/codes/mmdet-VPT/work_dirs/FLIR/Ablation/768_6chans_4-7/best_coco_bbox_mAP_iter_20500.pth'
# ch6 = torch.load(ch6, map_location='cpu')

vitv15_full = '/path/to/codes/mmdet-VPT/work_dirs/FLIR/Ablation/1024_fullfinetune_3-23/best_coco_bbox_mAP_iter_44500.pth'
vitv15_full = torch.load(vitv15_full, map_location='cpu')

/home/zhangguiwei/anaconda3/envs/detrex/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
vitv15_full_back = {k[9:]: v for k, v in vitv15_full['state_dict'].items() if 'backbone' in k}
vit_values = []
for k in vitv15_full_back.keys():
    if 'num_batches_tracked' not in k:  # batch norm 的参数, 不用考虑
        vit_values.append(vitv15_full_back[k])
# vitv15_full_back_params = [*vitv15_full_back.values()]
# vitv15_full_back_params[0]

In [8]:
adapter_back = {k[9:]: v for k, v in adapter_whole['state_dict'].items() if 'backbone' in k}
ch6_back = {k[9:]: v for k, v in ch6['state_dict'].items() if 'backbone' in k}

In [15]:
adapter_unfreeze = []
adapter_unfreeze_keys = []
for k in adapter_back.keys():
    if 'prompt' in k or 'adapter' in k:
        if 'num_batches_tracked' not in k:  # batch norm 的参数, 不用考虑
            adapter_unfreeze.append(adapter_back[k])
            adapter_unfreeze_keys.append(k)

ch6_unfreeze = [*ch6_back.values()]
ch6_unfreeze_keys = [*ch6_back.keys()]

In [16]:
len(adapter_unfreeze)

128

In [17]:
len(ch6_unfreeze)

171

In [8]:
def get_params(tensor_list):
    n_params = 0
    for i in tensor_list:
        shape = torch.tensor(i.shape)
        n_floats = torch.prod(shape)
        n_params += n_floats
    return n_params
        
# get_params(adapter_unfreeze), get_params(ch6_unfreeze) # 8.91M, 86.46M
get_params(vit_values) # 94.78M

tensor(94779088)

In [20]:
# 按照 part 保存参数(tensor)
head_tensors = []  # params except backbone, 包含 RPN head 和 RoI head
prompter_tensors = []
vit_tensors = []

for k in full_weight['state_dict']:
    tensor = full_weight['state_dict'][k]
    if 'backbone' not in k:
        head_tensors.append(tensor)
    else:
        if 'prompt' in k or 'adapter' in k:
            if 'num_batches_tracked' not in k:  # batch norm 的参数, 不用考虑
                prompter_tensors.append(tensor)
        else:
            vit_tensors.append(tensor)

In [21]:
len(head_tensors), len(prompter_tensors), len(vit_tensors)

(94, 128, 171)

In [28]:
n_floats_head = 0
n_floats_prompter = 0
n_floats_vit = 0

for i in head_tensors:
    shape = torch.tensor(i.shape)
    n_floats = torch.prod(shape)
    n_floats_head += n_floats.item()
    
for i in prompter_tensors:
    shape = torch.tensor(i.shape)
    n_floats = torch.prod(shape)
    n_floats_prompter += n_floats.item()
    
for i in vit_tensors:
    shape = torch.tensor(i.shape)
    n_floats = torch.prod(shape)
    n_floats_vit += n_floats.item()

In [29]:
n_floats_head, n_floats_prompter, n_floats_vit

(52390887, 8906448, 85889024)

In [30]:
n_floats_prompter / n_floats_vit  # only adding 10.37% learnable params to backbone.

0.10369716158376652

In [ ]:
# 全部参数: 147,186,359
# 放开参数占整个检测器比例(prompter+head): 41.65%
# 放开参数(prompter)占整个backbone(prompter+ViT)比例: 9.40%
# 放开参数(prompter)占ViT比例: 10.37%